### Importing the libraries

In [ ]:
import os,sys,kfp
from dkube.sdk import *

### Fill these values when you run the pipeline

In [ ]:
s3_access_key_id = ""
s3_access_key_secret = ""

### Dkube-SDK

In [ ]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
username =  os.getenv("USERNAME")
dkube_url = os.getenv("DKUBE_URL")
api = DkubeApi(URL=dkube_url,token=token)

### Pipeline Components

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)

### Creating a dataset in DKube with source as s3

In [ ]:
dataset = DkubeDataset(username, name="mnist")
dataset.update_dataset_source(source='s3')
dataset.update_awss3_details(bucket="dkube",prefix="",key = s3_access_key_id,secret = s3_access_key_secret)
dataset.datum.remote = True
api.create_dataset(dataset)

### Pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(token, program='training'):

    train       = dkube_training_op(token, '{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="dkube-examples", run_script="python mnist/train.py",
                                    datasets='["mnist"]', outputs='["mnist"]',
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]')

    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}',
                                    transformer_image='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    transformer_project="dkube-examples",
                                    transformer_code='mnist/transformer.py').after(train)


### Create a Run

In [ ]:
client.create_run_from_pipeline_func(mnist_pipeline, arguments={"token":token})